<a href="https://colab.research.google.com/github/Alejogesal/Proyecto1/blob/main/Actualizacion_N8n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Cómo crear una instancia de n8n a partir de una instantánea, actualizar su versión y realizar una copia de seguridad mediante instantáneas**

Esta guía proporciona instrucciones paso a paso para crear una instancia confiable de n8n, actualizar su versión y proteger sus datos mediante instantáneas. Siga cada paso con atención para garantizar un proceso sin inconvenientes.

**Documentación**

Para mayor referencia, consulte la documentación oficial: https://docs.n8n.io/hosting/installation/docker/#updating

1. Agregue su usuario al grupo Docker
Para ejecutar comandos Docker sin sudo:

In [ ]:
whoami

Te va a dar nombre de user

In [ ]:
sudo usermod -aG docker USER

Cierre sesión y vuelva a iniciarla para que los cambios surtan efecto:

In [ ]:
exit

2. Verifique la versión actual de **n8n**
Verifique la versión n8n que se ejecuta dentro del contenedor:

In [ ]:
docker exec -it n8n n8n --version

3. Inspeccione el contenedor para encontrar el montaje y el directorio correcto
Ejecute el siguiente comando para inspeccionar el contenedor n8n y localizar el punto de montaje correcto:

In [ ]:
docker inspect n8n

4. Actualizar el nombre de dominio (si ha cambiado)
Si el nombre de dominio de su instancia n8n ha cambiado, actualice su configuración de Nginx:

In [ ]:
sudo nano /etc/nginx/sites-available/n8n

Guarde los cambios con Ctrl + O, presione Entery luego salga con Ctrl + X.

5. Verificar el ID del contenedor
Enumere todos los contenedores para obtener el ID de contenedor correcto:

In [ ]:
docker ps -a

6. Copia de seguridad de los archivos de configuración desde el contenedor
Acceda al contenedor y enumere los archivos de configuración:

In [ ]:
docker exec -it --user node n8n sh
ls -la /home/node/.n8n

Salir del contenedor:



In [ ]:
exit

Realice una copia de seguridad de los archivos de configuración en su máquina local:

In [ ]:
docker cp <container_id>:/home/node/.n8n ./n8n_backup

Verifique el archivo de respaldo en su máquina local:

In [ ]:
ls -la ./n8n_backup

7. Actualizar n8n a una versión específica
Extraiga la versión deseada de la imagen Docker n8n:



In [ ]:
docker pull docker.n8n.io/n8nio/n8n:'VERSION'

8. Detener y retirar el contenedor existente
Detener y retirar el contenedor viejo:

In [ ]:
docker stop <container_id> se pone 'n8n'
docker rm <container_id> se pone 'n8n'

9. Cree un volumen Docker para datos persistentes9.
Cree un volumen Docker para datos persistentes

In [ ]:
docker volume create n8n_data

Copiar los datos de respaldo en el nuevo volumen:



In [ ]:
docker run --rm -v n8n_data:/data -v $(pwd)/n8n_backup:/backup busybox sh -c "cp -r /backup/* /data/"

Inspeccione el volumen para confirmar los datos:



In [ ]:
docker volume inspect n8n_data

10. Ajustar los permisos para el volumen
Asegúrese de que el nodeusuario dentro del contenedor tenga los permisos adecuados:



In [ ]:
sudo chown -R 1000:1000 /var/lib/docker/volumes/n8n_data/_data
sudo chmod -R 700 /var/lib/docker/volumes/n8n_data/_data

11. Cree un nuevo contenedor n8n con los datos restaurados
Ejecute un nuevo contenedor n8n con los datos restaurados "n8n_data" y la versión actualizada:



In [ ]:
docker run -d --restart unless-stopped \
--name n8n \
-p 5678:5678 \
-e N8N_HOST="your-domain.com" \
-e WEBHOOK_TUNNEL_URL="https://your-domain.com/" \
-e WEBHOOK_URL="https://your-domain.com/" \
-v n8n_data:/home/node/.n8n \  # here is where we mount n8n_data
n8nio/n8n:'version' # we call the latest stable version

12. Verificar los datos de respaldo en el contenedor
Inicie el contenedor y verifique que los datos se hayan restaurado correctamente:



In [ ]:
docker exec -it --user node n8n sh -c "ls -la /home/node/.n8n"

Información adicional importante (de lectura obligatoria)
Cómo funcionan los volúmenes persistentes
El n8n_datavolumen se almacena fuera del contenedor, lo que garantiza que todos los flujos de trabajo y configuraciones se conserven incluso si el contenedor se detiene o se elimina. Así es como funciona:

Los datos permanecen seguros al detener o reiniciar el contenedor :

Úselo docker stop n8npara detener la aplicación.
Úselo docker start n8npara reiniciarlo y sus datos permanecerán intactos.
El volumen persiste más allá de la eliminación del contenedor :

Si elimina el contenedor con docker rm n8n, los datos del n8n_datavolumen permanecerán seguros.
Puede reutilizar el volumen asignándolo a un nuevo contenedor:

In [ ]:
docker run -d --restart unless-stopped \
--name n8n \
-p 5678:5678 \
-e N8N_HOST="your-domain.com" \
-e WEBHOOK_TUNNEL_URL="https://your-domain.com/" \
-e WEBHOOK_URL="https://your-domain.com/" \
-v n8n_data:/home/node/.n8n \  # here is where we mount n8n_data
n8nio/n8n:latest # we call the latest stable version

Al aprovechar los volúmenes de Docker, garantiza que su instancia n8n siga siendo resistente y que sus datos estén seguros durante actualizaciones o cambios de contenedores.